# DuckDB and geoparquet

## Imports

In [1]:
import os
import duckdb
import folium
import pystac
import folium.plugins as folium_plugins
import geopandas as gpd
from stac_geoparquet.arrow import stac_table_to_items
from pygeofilter_duckdb import to_sql_where
from pygeofilter.util import IdempotentDict
from pygeofilter.parsers.cql2_json import parse as json_parse

In [2]:
# Install and load DuckDB spatial extension
duckdb.install_extension("spatial")
duckdb.load_extension("spatial")

In [3]:
s2_parquet_path = "euro.parquet"

## Query the geoparquet file

In [4]:
if not os.path.exists(s2_parquet_path):
    print("Run notebook 02-Create a geoparquet file.ipynb first")
    exit(1)

In [6]:
sql_query = f"SELECT * EXCLUDE(geometry), ST_AsWKB(geometry) as geometry FROM '{s2_parquet_path}'"

print(sql_query)

db = duckdb.query(sql_query)

SELECT * EXCLUDE(geometry), ST_AsWKB(geometry) as geometry FROM 'euro.parquet'


In [7]:
type(db)

duckdb.duckdb.DuckDBPyRelation

## Convert DuckDB result to Arrow table

In [15]:
s2_parquet_path = "euro.parquet"
sql_query = f"SELECT * EXCLUDE(geometry), ST_AsWKB(geometry) as geometry FROM 'euro.parquet' LIMIT 10;"

print(sql_query)

db = duckdb.query(sql_query)

table = db.fetch_arrow_table()

table_head = table.slice(0, 1)  # Get first 125 rows
for item in stac_table_to_items(table_head):
    display(item)

SELECT * EXCLUDE(geometry), ST_AsWKB(geometry) as geometry FROM 'euro.parquet' LIMIT 10;


GEOSException: ParseException: Input buffer is smaller than requested object size

## Map


In [11]:
for item in stac_table_to_items(table_head):
    display(item)

GEOSException: ParseException: Input buffer is smaller than requested object size

In [36]:
items = [pystac.Item.from_dict(item) for item in stac_table_to_items(table_head)]

map = folium.Map()
layer_control = folium.LayerControl(position="topright", collapsed=True)
fullscreen = folium_plugins.Fullscreen()
style = {"fillColor": "#00000000", "color": "#0000ff", "weight": 1}

footprints = folium.GeoJson(
    gpd.GeoDataFrame.from_features(items).to_json(),
    name="Stac Item footprints",
    style_function=lambda x: style,
    control=True,
)

footprints.add_to(map)
layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds())
map

GEOSException: ParseException: Input buffer is smaller than requested object size

### Geofilter

In [19]:
cql2_filter = {
    "op": "and",
    "args": [
        {
            "op": "between",
            "args": [
                {"property": "datetime"},
                ["2023-12-28T00:00:00Z", "2023-12-28T23:59:59Z"],
            ],
        },
        {"op": "between", "args": [{"property": "eo:cloud_cover"}, [90, 100]]},
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [-9.51346013858793, 38.95450355515311],
                            [-9.51346013858793, 38.22500810801125],
                            [-8.359265560322228, 38.22500810801125],
                            [-8.359265560322228, 38.95450355515311],
                            [-9.51346013858793, 38.95450355515311],
                        ]
                    ],
                },
            ],
        },
    ],
}

In [20]:
sql_where = to_sql_where(json_parse(cql2_filter), IdempotentDict())
print(sql_where)

((("datetime" BETWEEN '2023-12-28T00:00:00Z' AND '2023-12-28T23:59:59Z') AND ("eo:cloud_cover" BETWEEN 90 AND 100)) AND ST_Intersects("geometry",ST_GeomFromHEXEWKB('01030000000100000005000000D0114E3FE40623C030A6282C2D7A4340D0114E3FE40623C0209FD010CD1C43408053D0A7F1B720C0209FD010CD1C43408053D0A7F1B720C030A6282C2D7A4340D0114E3FE40623C030A6282C2D7A4340')))


In [21]:
sql_query = f"SELECT * EXCLUDE(geometry), ST_AsWKB(geometry) as geometry FROM '{s2_parquet_path}' WHERE {sql_where}"

In [22]:
db = duckdb.query(sql_query)

BinderException: Binder Error: Referenced column "eo:cloud_cover" not found in FROM clause!
Candidate bindings: "euro.proj:code", "euro.proj:geometry", "euro.collection", "euro.id", "euro.proj:bbox"

In [23]:
subset_table = db.fetch_arrow_table()
subset_table[0]

[
  -- is_valid: all not null
  -- child 0 type: struct<eo:bands: list<l: struct<center_wavelength: double, common_name: string, name: string>>, href: string, raster:bands: list<l: struct<histogram: struct<buckets: list<l: int64>, count: int64, max: double, min: double>, spatial_resolution: double, statistics: struct<maximum: double, mean: double, minimum: double, stddev: double, valid_percent: double>>>, roles: list<l: string>, title: string, type: string>
    -- is_valid: all not null
    -- child 0 type: list<l: struct<center_wavelength: double, common_name: string, name: string>>
      [
        -- is_valid: all not null
        -- child 0 type: double
          [
            0.443,
            0.49,
            0.56,
            0.665,
            0.705,
            ...
            0.865,
            0.945,
            1.375,
            1.61,
            2.19
          ]
        -- child 1 type: string
          [
            "coastal",
            "blue",
            "green",
  

In [24]:
items = [pystac.Item.from_dict(item) for item in stac_table_to_items(subset_table)]

items[0]

GEOSException: ParseException: Input buffer is smaller than requested object size

In [16]:
len(items)

4

In [25]:
map = folium.Map()
layer_control = folium.LayerControl(position="topright", collapsed=True)
fullscreen = folium_plugins.Fullscreen()
style = {"fillColor": "#00000000", "color": "#0000ff", "weight": 1}

footprints = folium.GeoJson(
    gpd.GeoDataFrame.from_features(items).to_json(),
    name="Stac Item footprints",
    style_function=lambda x: style,
    control=True,
)

footprints.add_to(map)
layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds())
map

NameError: name 'items' is not defined